In [1]:
!gdown --i 1sRELpaE3GICTBvm9Pb9_cTc6y4GfTovt

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1sRELpaE3GICTBvm9Pb9_cTc6y4GfTovt
To: /content/full_data_125k.csv
100% 82.9M/82.9M [00:02<00:00, 38.6MB/s]


In [2]:
!pip install transformers seqeval[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=2bffd002d6f926903ce194ecf70e2fd2a121f84236f4d7f42b40e59950669bd8
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [5]:
data = pd.read_csv('/content/full_data_125k.csv')
data

,Unnamed: 0,sentence,word_labels
0,0,This includes reviewing large numbers of docum...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In..."
1,1,Will report directly to Head of Weather Resear...,"O,O,O,O,O,O,O,In-SK,O,O,O,O,O"
2,2,The analyst will be expected to interpret resu...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In..."
3,3,A wide degree of creativity and latitude is ex...,"O,O,O,O,In-SK,O,O,O,O"
4,4,You must be authorized to work in the United S...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,..."
...,...,...,...
124399,124399,Simpson Strong Tie s drug policy is based upon...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
124400,124400,We believe all employees affect customers sal...,"O,O,O,O,O,O,In-SK,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
124401,124401,In keeping with the Company s commitment to th...,"O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,O,O,O,O,O,O,O"
124402,124402,Complete training as required to ensure you ar...,"O,In-SK,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"


In [6]:
data.dropna(inplace=True)
data

,Unnamed: 0,sentence,word_labels
0,0,This includes reviewing large numbers of docum...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In..."
1,1,Will report directly to Head of Weather Resear...,"O,O,O,O,O,O,O,In-SK,O,O,O,O,O"
2,2,The analyst will be expected to interpret resu...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In..."
3,3,A wide degree of creativity and latitude is ex...,"O,O,O,O,In-SK,O,O,O,O"
4,4,You must be authorized to work in the United S...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,..."
...,...,...,...
124399,124399,Simpson Strong Tie s drug policy is based upon...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
124400,124400,We believe all employees affect customers sal...,"O,O,O,O,O,O,In-SK,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
124401,124401,In keeping with the Company s commitment to th...,"O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,O,O,O,O,O,O,O"
124402,124402,Complete training as required to ensure you ar...,"O,In-SK,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"


In [7]:
labels = ['O', 'In-SK']

In [8]:
label2id = {k: v for v, k in enumerate(labels)}
id2label = {v: k for v, k in enumerate(labels)}
label2id

{'O': 0, 'In-SK': 1}

In [9]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data

,sentence,word_labels
0,This includes reviewing large numbers of docum...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In..."
1,Will report directly to Head of Weather Resear...,"O,O,O,O,O,O,O,In-SK,O,O,O,O,O"
2,The analyst will be expected to interpret resu...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In..."
3,A wide degree of creativity and latitude is ex...,"O,O,O,O,In-SK,O,O,O,O"
4,You must be authorized to work in the United S...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,..."
...,...,...
124399,Simpson Strong Tie s drug policy is based upon...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
124400,We believe all employees affect customers sal...,"O,O,O,O,O,O,In-SK,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
124401,In keeping with the Company s commitment to th...,"O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,O,O,O,O,O,O,O"
124402,Complete training as required to ensure you ar...,"O,In-SK,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"


In [10]:
len(data)

124404

In [17]:
data.iloc[41].sentence

'Proficient in MS Office  Word  Excel  Access  PowerPoint  PreferredTen  10   years  Aviation Maintenance experience Seven  7   years  experience on fleet type relevant to the assigned program Seven  7   years  experience working government contract service Have had at least one tour in the USAF and have been a C 130H Crew Chief Through knowledge and understanding of applicable C 130 Maintenance Program'

In [18]:
data.iloc[41].word_labels

'O,O,O,O,O,In-SK,O,In-SK,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,In-SK,O,O,O,O,O,O,O,O,O,In-SK,O,O,O'

#### **Preparing the dataset and dataloader**

In [29]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 4
EPOCHS = 3
LEARNING_RATE = 4e-06
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [30]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [31]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [32]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (124404, 2)
TRAIN Dataset: (99523, 2)
TEST Dataset: (24881, 2)


Let's have a look at the first training example:

In [33]:
training_set[0]

{'ids': tensor([  101,  2057,  2024,  7568,  2000,  2424,  9788,  2136,  2372,  2000,
          3693,  2149,  2006,  2023,  4990,  3105,  6412,  5114,  3993,  2003,
         14763,  1037,  2951, 12941,  2000,  2147,  2007,  3330,  1998,  3930,
          2780,  1999,  2344,  2000,  2855,  2009, 22139,  2256,  4031,  2119,
          3558,  1998,  3617,  1999,  2344,  2000,  5147,  3113,  2256,  6304,
          3791,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

Let's verify that the input ids and corresponding targets are correct:

In [34]:
training_set[0]["ids"]

tensor([  101,  2057,  2024,  7568,  2000,  2424,  9788,  2136,  2372,  2000,
         3693,  2149,  2006,  2023,  4990,  3105,  6412,  5114,  3993,  2003,
        14763,  1037,  2951, 12941,  2000,  2147,  2007,  3330,  1998,  3930,
         2780,  1999,  2344,  2000,  2855,  2009, 22139,  2256,  4031,  2119,
         3558,  1998,  3617,  1999,  2344,  2000,  5147,  3113,  2256,  6304,
         3791,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [35]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
we          O
are         O
excited     O
to          O
find        O
incredible  O
team        O
members     O
to          O
join        O
us          O
on          O
this        O
journey     O
job         O
description  O
gain        O
##ful       O
is          O
hiring      In-SK
a           O
data        O
analyst     O
to          O
work        O
with        O
engineering  In-SK
and         O
growth      O


Now, let's define the corresponding PyTorch dataloaders:

In [36]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

In [37]:
model = BertForTokenClassification.from_pretrained('distilbert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.layer.11.attention.output.LayerNorm.weight', 'encoder.layer.3.intermediate.dense.weight', 'encoder.layer.6.output.LayerNorm.weight', 'encoder.layer.5.attention.output.dense.weight', 'encoder.layer.1.attention.self.value.weight', 'encoder.layer.9.attention.self.query.weight', 'encoder.layer.5.attention.self.value.bias', 'encoder.layer.3.attention.self.key.bias', 'encoder.layer.0.output.dense.bias', 'encoder.layer.2.attention.self.key.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.6.attention.self.query.weight', 'encoder.layer.1.attention.self.key.weight', 'encoder.layer.4.output.LayerNorm.bias', 'encoder.layer.9.intermediate.dense.bias', 'encoder.layer.8.a

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [38]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.0828, device='cuda:0', grad_fn=<NllLossBackward0>)

In [39]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 2])

In [40]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [41]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [42]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.0638011693954468
Training loss per 100 training steps: 0.16489523531186698
Training loss per 100 training steps: 0.14187829207573363
Training loss per 100 training steps: 0.12529159454857788
Training loss per 100 training steps: 0.11473776710375289
Training loss per 100 training steps: 0.1061104261827564
Training loss per 100 training steps: 0.09991501918226828
Training loss per 100 training steps: 0.09501638037342573
Training loss per 100 training steps: 0.09091162443226122
Training loss per 100 training steps: 0.08766459811325872
Training loss per 100 training steps: 0.08484774187475175
Training loss per 100 training steps: 0.0824287613493464
Training loss per 100 training steps: 0.07995030714150869
Training loss per 100 training steps: 0.0780119624570138
Training loss per 100 training steps: 0.0763931434792995
Training loss per 100 training steps: 0.07501486938280474
Training loss per 100 training steps: 0.07354284907987273
T

#### **Evaluating the model**

Now that we've trained our model, we can evaluate its performance on the held-out test set (which is 20% of the data). Note that here, no gradient updates are performed, the model just outputs its logits.

In [43]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

As we can see below, performance is quite good! Accuracy on the test test is > 97%.

In [44]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.051613204181194305
Validation loss per 100 evaluation steps: 0.029850522013574764
Validation loss per 100 evaluation steps: 0.02843243277698885
Validation loss per 100 evaluation steps: 0.028430009856000173
Validation loss per 100 evaluation steps: 0.0285120880879892
Validation loss per 100 evaluation steps: 0.029614319421771305
Validation loss per 100 evaluation steps: 0.030063811734287935
Validation loss per 100 evaluation steps: 0.029825838928489706
Validation loss per 100 evaluation steps: 0.030305853644976732
Validation loss per 100 evaluation steps: 0.030678467663627383
Validation loss per 100 evaluation steps: 0.030538079952619358
Validation loss per 100 evaluation steps: 0.030666969157271632
Validation loss per 100 evaluation steps: 0.030503808385671696
Validation loss per 100 evaluation steps: 0.03066454383336893
Validation loss per 100 evaluation steps: 0.03076260081348985
Validation loss per 100 evaluation steps: 0.0305489404813491

However, the accuracy metric is misleading, as a lot of labels are "outside" (O), even after omitting predictions on the [PAD] tokens. What is important is looking at the precision, recall and f1-score of the individual tags. For this, we use the seqeval Python library:

In [45]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: In-SK seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

          SK       0.69      0.81      0.75     78270

   micro avg       0.69      0.81      0.75     78270
   macro avg       0.69      0.81      0.75     78270
weighted avg       0.69      0.81      0.75     78270



#### **Inference**

The fun part is when we can quickly test the model on new, unseen sentences.
Here, we use the prediction of the **first word piece of every word**. Note that the function we used to train our model (`tokenze_and_preserve_labels`) propagated the label to all subsequent word pieces (so you could for example also perform a majority vote on the predicted labels of all word pieces of a word).

*In other words, the code below does not take into account when predictions of different word pieces that belong to the same word do not match.*

In [47]:
sentence = "Searching for new opportunities as Junior Node.js JavaScript backend developer. Over 15 years of experience in different IT areas. Experience with: Node.js JavaScript MongoDB  HTML CSS Java Lotus Script websocket socket.io Docker babel Webpack MySQL JSON React"
inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

word_level_predictions = []
for pair in wp_preds:
  if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
    # skip prediction
    continue
  else:
    word_level_predictions.append(pair[1])

# we join tokens, if they are not special ones
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
print(str_rep)
print(word_level_predictions)

searching for new opportunities as junior node . js javascript backend developer . over 15 years of experience in different it areas . experience with : node . js javascript mongodb html css java lotus script websocket socket . io docker babel webpack mysql json react
['O', 'O', 'O', 'O', 'O', 'O', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'O', 'O', 'O', 'In-SK', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'In-SK', 'O', 'O', 'O', 'O', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'O', 'In-SK', 'In-SK', 'O', 'O', 'In-SK', 'O', 'O', 'In-SK', 'In-SK', 'In-SK', 'O', 'In-SK', 'In-SK', 'O', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK', 'In-SK']


Note that there's another way to easily perform quick inference with a trained model: the [pipeline API](https://huggingface.co/docs/transformers/main_classes/pipelines). The pipeline API abstracts away all the complexity for you (basically performing what we did above). Here, we'll use the [TokenClassificationPipeline](https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/pipelines#transformers.TokenClassificationPipeline) since that's the task we're doing, and we provide a model and tokenizer.

In [48]:
from transformers import pipeline


pipe = pipeline(task="token-classification", model=model.to("cpu"), tokenizer=tokenizer, aggregation_strategy="simple")
pipe(sentence)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'SK',
  'score': 0.70664907,
  'word': 'node . js javascript',
  'start': None,
  'end': None},
 {'entity_group': 'SK',
  'score': 0.58070785,
  'word': '.',
  'start': None,
  'end': None},
 {'entity_group': 'SK',
  'score': 0.8224279,
  'word': 'it',
  'start': None,
  'end': None},
 {'entity_group': 'SK',
  'score': 0.719559,
  'word': ': node . js javascript mongodb html css java',
  'start': None,
  'end': None},
 {'entity_group': 'SK',
  'score': 0.7036481,
  'word': 'script web',
  'start': None,
  'end': None},
 {'entity_group': 'SK',
  'score': 0.583883,
  'word': '##t',
  'start': None,
  'end': None},
 {'entity_group': 'SK',
  'score': 0.7480984,
  'word': 'io docker',
  'start': None,
  'end': None},
 {'entity_group': 'SK',
  'score': 0.7309417,
  'word': '##l web',
  'start': None,
  'end': None},
 {'entity_group': 'SK',
  'score': 0.8219047,
  'word': 'mysql json react',
  'start': None,
  'end': None}]

#### **Saving the model for future use**

Finally, let's save the model and tokenizer files such that we can easily re-use them later on. There are 2 options:

* you can save everything locally, simply by calling `model.save_pretrained()` and `tokenizer.save_pretrained()`, providing a directory path as argument.
* you can push the files to the [HuggingFace hub](https://huggingface.co/). This way, you can share your model with the community/your colleagues. All files will be tracked by git, as each model on the hub has its own git repo.

Both options allow to re-use the model/tokenizer using the `from_pretrained()` method. Here we'll do the latter.

To upload a model to the hub, 2 things need to be setup:
* install git-LFS, which is used by the hub
* set up authentication token


In [49]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [50]:
from huggingface_hub import notebook_login

notebook_login()

In [51]:
model_name = "skill-role-mapper"

# upload files to the hub
tokenizer.push_to_hub(
    repo_id=model_name,
    organization="MehdiHosseiniMoghadam",
    commit_message="Add tokenizer",
    use_temp_dir=True,
)
model.push_to_hub(
    repo_id=model_name,
    organization="MehdiHosseiniMoghadam",
    commit_message="Add model",
    use_temp_dir=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:700: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MehdiHosseiniMoghadam/skill-role-mapper/commit/567c698aaa96f6271d0a3301c3ffaeca249990c7', commit_message='Add model', commit_description='', oid='567c698aaa96f6271d0a3301c3ffaeca249990c7', pr_url=None, pr_revision=None, pr_num=None)